In [ ]:
# read in extracted target regions post-alignment and parse to identify edits within 
# user-defined target windows

In [ ]:
import sys
import pandas as pd

In [ ]:
# now read in the aligned samples and:
# (1) dict of lists with barcodes identifying groups as the key and list of CIGARs as values

import os
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm

def read_aligns(input_dir, bc_start, bc_end):
    
    index_lst = []
    
    master_dict = defaultdict(dict)
    
    for subdir, dirs, files in os.walk(input_dir):
                
        if len(files) != 0:
            
            for file in tqdm(files):
                
                if "needleall" in file:
                    full_path = subdir + "/" + file
                    #print(full_path)
                    
                    sample_name = file.split(".")[0]
                    #print(sample_name)
                    
                    sample_dict = defaultdict(list)
                    
                    with open(full_path) as align:
                    
                        line_1 = align.readline()
                        line_2 = align.readline()
                        line_3 = align.readline()
                    
                        for line in tqdm(align):
                        
                            #print(line)
                            line_rstrip = line.rstrip('\n')
                            line_split = line_rstrip.split('\t')
                            read_name = line_split[0]
                            cigar = line_split[5]
                            read = line_split[9]
                            bc = read[bc_start:bc_end]
                            sample_dict[bc].append(cigar)
                            
                        master_dict[sample_name].update(sample_dict)
            
    print(master_dict)
    
    return master_dict
    
in_align = read_aligns("user defined in dir", 7, 14)


In [ ]:
# now loop through master dictionary and calculate on-target edit efficiency

import re
import pandas as pd
import statistics

def calc_edit(in_dict, cutoff):
    
    filter_match = r'^(\d+)[M]{1}(\d+[MID]{1})+(\d+)[M]{1}$'
    univ_string = r'(\d+)([MID]{1})'
    
    out_dict = {} # list of series with samples as keys
    
    for sample, dct in tqdm(in_dict.items()):
        
        edit_sries = pd.Series()
        tot_read_sries = pd.Series()
        
        read_rep_lst = []
        
        for bc, lst in dct.items():
            
            #print(tam)
            
            num_reads = len(lst)
            read_rep_lst.append(num_reads)
            edited_reads = 0
            allele_id = []
            
            if num_reads >= cutoff:
                
                for cigar in lst:

                    parse = re.findall(univ_string, cigar)
                    i = 0 # init amplicon at 5' end
                    amp_index_lst = []
                    j = 0

                    for element in parse:

                        edit_len = int(element[0])
                        edit_type = element[1]

                        if edit_type == "M" or edit_type == "D":

                            i += edit_len

                            amp_index_lst.append(i)

                        if edit_type == "D":

                            indl_bgin = i - edit_len

                            if i in range(29, 34) and edit_len > 0: # indexes depend on amplicon design

                                edited_reads += 1
                                allele_id.append(cigar)
                                
                                break

                            elif indl_bgin in range(29, 34) and edit_len > 0: # indexes depend on amplicon design

                                edited_reads += 1
                                allele_id.append(cigar)
                                
                                break

                edit_rate = round(edited_reads / num_reads, 3)
                
                edit_sries[bc] = edit_rate
                tot_read_sries[bc] = num_reads
                
        med = statistics.median(read_rep_lst)
        mean = statistics.mean(read_rep_lst)

        out_dict[sample + "_edit_rate"] = edit_sries
        out_dict[sample + "_tot_reads"] = tot_read_sries
        
    return out_dict
                                            
calc_1 = calc_edit(in_align, 0)
                
        
        
        
        
    
    
    
    